In [2]:
#importing the package
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk import TreebankWordTokenizer, SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from emoji import UNICODE_EMOJI
import emoji as emoj
import demoji
from gensim import models
from gensim.models import word2vec

#### Download and install the Following

In [26]:
#nltk.download('wordnet')
#nltk.download('stopwords')
#!pip install emoji
#!pip install demoji
#demoji.download_codes()
#demoji.download_codes()
# pip install plotly 
#install genism on the ec2 instance import subprocess then , subprocess.check_call(["python",'-m','pip','install','gensim'])

In [3]:
#importing the data
data = pd.read_csv('train.csv')
data1 = data
data.head()

,sentiment,message,tweetid
0,-1,RT @darreljorstad: Funny as hell! Canada deman...,897853122080407553
1,-1,All the biggest lies about climate change and ...,925046776553529344
2,-1,The Coming Revelation Of The $q$Global Warming...,696354236850786305
3,-1,RT @DineshDSouza: Let's see if the world ends ...,846806509732483072
4,-1,RT @SteveSGoddard: Obama has no control over t...,628085266293653504


In [3]:
#checking columns
data.columns

Index(['sentiment', 'message', 'tweetid'], dtype='object')

In [4]:
data.shape

(30759, 3)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30759 entries, 0 to 30758
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  30759 non-null  int64 
 1   message    30759 non-null  object
 2   tweetid    30759 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 721.0+ KB


In [6]:
data.describe()

,sentiment,tweetid
count,30759.000000,3.075900e+04
mean,0.853929,8.364330e+17
std,0.853551,8.519986e+16
min,-1.000000,5.926334e+17
25%,0.000000,7.970301e+17
50%,1.000000,8.401516e+17
75%,1.000000,8.996975e+17
max,2.000000,9.666876e+17


In [7]:
data[['message']].describe()

,message
count,30759
unique,28813
top,RT @StephenSchlegel: she's thinking about how ...
freq,361


In [8]:
data.nunique()

sentiment        4
message      28813
tweetid      30759
dtype: int64

### Data Cleaning

### Finding Rows with emoji

In [21]:
def text_has_emoji(text):
    """This function checks all the rows of data to see if we have any emojis in the """
    for character in text:
        if character in emoj.UNICODE_EMOJI:
            return True
    return False
data['emoji'] = data['message'].apply(text_has_emoji)


We see below that the message column has 1015 rows that have emojis

In [22]:
data[data['emoji'] == True]

,sentiment,message,tweetid,emoji,emojis


#### Extract All The Emojis In The Dataframe

In [12]:
def extra_all_the_emoji(strings):
    """We extract the emojis for every row in the dataframe"""
    return ''.join(character for character in strings if character in emoj.UNICODE_EMOJI)

We apply the extra_all_the_emoji function to the message column

In [13]:
"""We see all the emojis in the dataframe"""
data['emojis'] = data['message'].apply(extra_all_the_emoji)
data[data['emojis'] != '']['emojis']

7           ☀
25         ™™
44         ™™
71          ™
107        🇺🇸
         ... 
30354       ™
30595    ♥♥♥♥
30659     ™™™
30682       ™
30751       ⚡
Name: emojis, Length: 1015, dtype: object

In [14]:
"""The list of all emojis"""
list_of_emojis = data[data['emojis'] != '']['emojis'].tolist()

In [17]:
"""We loop through the whole list to change every emoji to text """
changed_emoji_to_text = []
for emojis in list_of_emojis:
    changed_emoji_to_text.append(emoj.demojize(emojis, delimiters=("", "")))
changed_emoji_to_text    

['sun',
 'trade_marktrade_mark',
 'trade_marktrade_mark',
 'trade_mark',
 'United_States',
 'trade_mark',
 'face_with_tears_of_joyface_with_tears_of_joy',
 'United_StatesUnited_States',
 'zipper-mouth_facezipper-mouth_face',
 'party_popperthinking_faceface_with_tears_of_joyface_with_tears_of_joy',
 'trade_mark',
 'trade_markcopyright',
 'fire',
 'trade_mark',
 'squinting_face_with_tongueface_with_tongueface_with_tongue',
 'trade_mark',
 'trade_mark',
 'trade_mark',
 'oncoming_fist_light_skin_tonemiddle_finger_light_skin_tone',
 'trade_marktrade_mark',
 'trade_mark',
 'trade_mark',
 'trade_mark',
 'sparklersparkler',
 'trade_mark',
 'trade_mark',
 'trade_marktrade_mark',
 'trade_marktrade_mark',
 'trade_mark',
 'trade_mark',
 'trade_mark',
 'trade_mark',
 'trade_mark',
 'trade_mark',
 'trade_mark',
 'trade_mark',
 'trade_mark',
 'trade_mark',
 'snowflake',
 'trade_mark',
 'nail_polish_medium_skin_tone',
 'left_arrow',
 'trade_marktrade_mark',
 'grinning_facegrinning_face_with_sweat',
 '

Now since we changed the emojis to text in a list we change them in the dataframe using the function below

In [20]:
def emojis_to_text(text):
    """We change all the emojis in the message column into words"""
    return emoj.demojize(text, delimiters=("", ""))
data['message'] = data['message'].apply(emojis_to_text)

#### Removing Twitter Handles

In [23]:
def remove_twitter_handles(tweet, pattern):
    """This function is used to remove twitter handles"""
    r = re.findall(pattern, tweet)
    for text in r:
        tweet = re.sub(text, '', tweet)
    return tweet

data['clean_tweet'] = np.vectorize(remove_twitter_handles)(data['message'], "@[\w]*") 

In [24]:
data.head()

,sentiment,message,tweetid,emoji,emojis,clean_tweet
0,-1,RT @darreljorstad: Funny as hell! Canada deman...,897853122080407553,False,,RT : Funny as hell! Canada demands 'gender rig...
1,-1,All the biggest lies about climate change and ...,925046776553529344,False,,All the biggest lies about climate change and ...
2,-1,The Coming Revelation Of The $q$Global Warming...,696354236850786305,False,,The Coming Revelation Of The $q$Global Warming...
3,-1,RT @DineshDSouza: Let's see if the world ends ...,846806509732483072,False,,RT : Let's see if the world ends when 's clim...
4,-1,RT @SteveSGoddard: Obama has no control over t...,628085266293653504,False,,RT : Obama has no control over the climate. He...


We use value counts to see if we stil have any emojis in the dataframe

In [33]:
data['emoji'].value_counts()

False    30759
Name: emoji, dtype: int64

Removing Stopwords From the DataFrame

In [34]:
stop_words = nltk.corpus.stopwords.words('english')

In [35]:
data['tidy_tweet'] = data['clean_tweet'].apply(lambda x: ' '.join([w for w in x.split() if w not in stop_words]))

In [36]:
data.head()

,sentiment,message,tweetid,emoji,emojis,clean_tweet,tidy_tweet
0,-1,RT @darreljorstad: Funny as hell! Canada deman...,897853122080407553,False,,RT : Funny as hell! Canada demands 'gender rig...,RT : Funny hell! Canada demands 'gender rights...
1,-1,All the biggest lies about climate change and ...,925046776553529344,False,,All the biggest lies about climate change and ...,All biggest lies climate change global warming...
2,-1,The Coming Revelation Of The $q$Global Warming...,696354236850786305,False,,The Coming Revelation Of The $q$Global Warming...,The Coming Revelation Of The $q$Global Warming...
3,-1,RT @DineshDSouza: Let's see if the world ends ...,846806509732483072,False,,RT : Let's see if the world ends when 's clim...,RT : Let's see world ends 's climate change ro...
4,-1,RT @SteveSGoddard: Obama has no control over t...,628085266293653504,False,,RT : Obama has no control over the climate. He...,RT : Obama control climate. He worst snake oil...


Remove http URL from data

In [37]:
data['tidy_tweet'] = data['tidy_tweet'].apply(lambda x: re.split('https:\/\/.*',str(x))[0])
data['tidy_tweet'] 

0        RT : Funny hell! Canada demands 'gender rights...
1        All biggest lies climate change global warming...
2        The Coming Revelation Of The $q$Global Warming...
3        RT : Let's see world ends 's climate change ro...
4        RT : Obama control climate. He worst snake oil...
                               ...                        
30754    RT : The Pentagon warned climate change threat...
30755    Study finds global warming exacerbates refugee...
30756    RT : The global green movement prepares fight ...
30757    RT : NYC Mayor says NY go faster address clima...
30758         RT Sanders: Climate Change Causes Terrorism 
Name: tidy_tweet, Length: 30759, dtype: object

Text Normalization

In [38]:
def tokenizing(text):
    text = re.split('\W+', text)
    return text

data['tokenized_tweet'] = data['tidy_tweet'].apply(lambda x: tokenizing(x))

In [39]:
data.head()

,sentiment,message,tweetid,emoji,emojis,clean_tweet,tidy_tweet,tokenized_tweet
0,-1,RT @darreljorstad: Funny as hell! Canada deman...,897853122080407553,False,,RT : Funny as hell! Canada demands 'gender rig...,RT : Funny hell! Canada demands 'gender rights...,"[RT, Funny, hell, Canada, demands, gender, rig..."
1,-1,All the biggest lies about climate change and ...,925046776553529344,False,,All the biggest lies about climate change and ...,All biggest lies climate change global warming...,"[All, biggest, lies, climate, change, global, ..."
2,-1,The Coming Revelation Of The $q$Global Warming...,696354236850786305,False,,The Coming Revelation Of The $q$Global Warming...,The Coming Revelation Of The $q$Global Warming...,"[The, Coming, Revelation, Of, The, q, Global, ..."
3,-1,RT @DineshDSouza: Let's see if the world ends ...,846806509732483072,False,,RT : Let's see if the world ends when 's clim...,RT : Let's see world ends 's climate change ro...,"[RT, Let, s, see, world, ends, s, climate, cha..."
4,-1,RT @SteveSGoddard: Obama has no control over t...,628085266293653504,False,,RT : Obama has no control over the climate. He...,RT : Obama control climate. He worst snake oil...,"[RT, Obama, control, climate, He, worst, snake..."


In [40]:
tokens = data['tokenized_tweet']

In [41]:
lemmatizer = WordNetLemmatizer()

In [42]:
tokens = tokens.apply(lambda x: [lemmatizer.lemmatize(i) for i in x])

In [43]:
data['lemmatized_tweet'] = tokens

In [44]:
data.head()

,sentiment,message,tweetid,emoji,emojis,clean_tweet,tidy_tweet,tokenized_tweet,lemmatized_tweet
0,-1,RT @darreljorstad: Funny as hell! Canada deman...,897853122080407553,False,,RT : Funny as hell! Canada demands 'gender rig...,RT : Funny hell! Canada demands 'gender rights...,"[RT, Funny, hell, Canada, demands, gender, rig...","[RT, Funny, hell, Canada, demand, gender, righ..."
1,-1,All the biggest lies about climate change and ...,925046776553529344,False,,All the biggest lies about climate change and ...,All biggest lies climate change global warming...,"[All, biggest, lies, climate, change, global, ...","[All, biggest, lie, climate, change, global, w..."
2,-1,The Coming Revelation Of The $q$Global Warming...,696354236850786305,False,,The Coming Revelation Of The $q$Global Warming...,The Coming Revelation Of The $q$Global Warming...,"[The, Coming, Revelation, Of, The, q, Global, ...","[The, Coming, Revelation, Of, The, q, Global, ..."
3,-1,RT @DineshDSouza: Let's see if the world ends ...,846806509732483072,False,,RT : Let's see if the world ends when 's clim...,RT : Let's see world ends 's climate change ro...,"[RT, Let, s, see, world, ends, s, climate, cha...","[RT, Let, s, see, world, end, s, climate, chan..."
4,-1,RT @SteveSGoddard: Obama has no control over t...,628085266293653504,False,,RT : Obama has no control over the climate. He...,RT : Obama control climate. He worst snake oil...,"[RT, Obama, control, climate, He, worst, snake...","[RT, Obama, control, climate, He, worst, snake..."


In [45]:
data = data.drop(['tidy_tweet'],axis=1)
data = data.drop(['tokenized_tweet'], axis=1)

In [46]:
data.head()

,sentiment,message,tweetid,emoji,emojis,clean_tweet,lemmatized_tweet
0,-1,RT @darreljorstad: Funny as hell! Canada deman...,897853122080407553,False,,RT : Funny as hell! Canada demands 'gender rig...,"[RT, Funny, hell, Canada, demand, gender, righ..."
1,-1,All the biggest lies about climate change and ...,925046776553529344,False,,All the biggest lies about climate change and ...,"[All, biggest, lie, climate, change, global, w..."
2,-1,The Coming Revelation Of The $q$Global Warming...,696354236850786305,False,,The Coming Revelation Of The $q$Global Warming...,"[The, Coming, Revelation, Of, The, q, Global, ..."
3,-1,RT @DineshDSouza: Let's see if the world ends ...,846806509732483072,False,,RT : Let's see if the world ends when 's clim...,"[RT, Let, s, see, world, end, s, climate, chan..."
4,-1,RT @SteveSGoddard: Obama has no control over t...,628085266293653504,False,,RT : Obama has no control over the climate. He...,"[RT, Obama, control, climate, He, worst, snake..."


In [47]:
clean_data = data[['sentiment','message','lemmatized_tweet','clean_tweet','tweetid']]

In [48]:
clean_data.head()

,sentiment,message,lemmatized_tweet,clean_tweet,tweetid
0,-1,RT @darreljorstad: Funny as hell! Canada deman...,"[RT, Funny, hell, Canada, demand, gender, righ...",RT : Funny as hell! Canada demands 'gender rig...,897853122080407553
1,-1,All the biggest lies about climate change and ...,"[All, biggest, lie, climate, change, global, w...",All the biggest lies about climate change and ...,925046776553529344
2,-1,The Coming Revelation Of The $q$Global Warming...,"[The, Coming, Revelation, Of, The, q, Global, ...",The Coming Revelation Of The $q$Global Warming...,696354236850786305
3,-1,RT @DineshDSouza: Let's see if the world ends ...,"[RT, Let, s, see, world, end, s, climate, chan...",RT : Let's see if the world ends when 's clim...,846806509732483072
4,-1,RT @SteveSGoddard: Obama has no control over t...,"[RT, Obama, control, climate, He, worst, snake...",RT : Obama has no control over the climate. He...,628085266293653504


In [49]:
clean_data.to_csv("clean_data.csv",index=False)

### Visualizing Tweets with Word2Vec and t-SNE

In [4]:
data_v = data1.copy()
data_v.head()

,sentiment,message,tweetid
0,-1,RT @darreljorstad: Funny as hell! Canada deman...,897853122080407553
1,-1,All the biggest lies about climate change and ...,925046776553529344
2,-1,The Coming Revelation Of The $q$Global Warming...,696354236850786305
3,-1,RT @DineshDSouza: Let's see if the world ends ...,846806509732483072
4,-1,RT @SteveSGoddard: Obama has no control over t...,628085266293653504


In [5]:
STOP_WORDS = nltk.corpus.stopwords.words()

def clean_sentence(val):
    "remove chars that are not letters or numbers, downcase, then remove stop words"
    regex = re.compile('([^\s\w]|_)+')
    sentence = regex.sub('', val).lower()
    sentence = sentence.split(" ")
    
    for word in list(sentence):
        if word in STOP_WORDS:
            sentence.remove(word)  
            
    sentence = " ".join(sentence)
    return sentence

def clean_dataframe(data):
    "drop nans, then apply 'clean_sentence' function to question1 and 2"
    data = data.dropna(how="any")
    
    for col in ['message']:
        data[col] = data[col].apply(clean_sentence)
    
    return data
data_v = clean_dataframe(data_v)
data_v.head(5)

,sentiment,message,tweetid
0,-1,rt darreljorstad funny hell canada demands gen...,897853122080407553
1,-1,biggest lies climate change global warming deb...,925046776553529344
2,-1,coming revelation qglobal warmingq fraud resem...,696354236850786305
3,-1,rt dineshdsouza lets see world ends realdonald...,846806509732483072
4,-1,rt stevesgoddard obama control climate worst s...,628085266293653504


In [6]:
def build_corpus(data):
    "Creates a list of lists containing words from each sentence"
    corpus = []
    for col in ['message']:
        for sentence in data[col].iteritems():
            word_list = sentence[1].split(" ")
            corpus.append(word_list)
            
    return corpus

corpus = build_corpus(data_v)        
corpus[0:2]

[['rt',
  'darreljorstad',
  'funny',
  'hell',
  'canada',
  'demands',
  'gender',
  'rights',
  'climate',
  'change',
  'trade',
  'deal',
  'soviet',
  'dairy',
  'boards',
  'untouc'],
 ['biggest',
  'lies',
  'climate',
  'change',
  'global',
  'warming',
  'debunked',
  'astonishing',
  'interview',
  'httpstcongvipo4wya']]

In [7]:
model = word2vec.Word2Vec(corpus, size=100, window=20, min_count=200, workers=4)
model.wv['warming']

array([-0.08403151, -0.43267834, -0.55697215, -0.07822817,  0.1184936 ,
        0.42608762,  0.09344044,  0.23876254,  0.03202659, -0.48474556,
        0.20207682,  0.3591411 ,  0.14153151, -0.59158474,  0.22894031,
       -0.50379354,  0.26142266,  0.13090676,  0.2817624 ,  0.18732384,
       -0.70734394, -0.17352863, -0.41382965, -0.07756615, -0.27795506,
        0.29656872, -0.2805533 , -0.14593855, -0.26067066, -0.3245186 ,
        0.06118457, -0.43256775, -0.01767822, -0.2653519 , -0.36534983,
        0.6677341 ,  0.06900666,  0.1283574 ,  0.55653936,  0.3253815 ,
       -0.6583108 ,  0.3805514 ,  0.22883272,  0.03484762,  0.16642113,
        0.1955169 ,  0.19038297, -0.45978984,  0.5591574 ,  0.20669614,
       -0.25710267, -0.63774097, -0.19494958,  0.2651378 ,  0.06040186,
        0.28029934, -0.19887646, -0.41886365,  0.49031952, -0.3546757 ,
       -0.3874937 , -0.2980653 ,  0.46184003,  0.48511234,  0.3602015 ,
        0.01042802, -0.24521618,  0.9268969 ,  0.05290376, -0.50

In [35]:
model.wv['warming']

array([-0.08403151, -0.43267834, -0.55697215, -0.07822817,  0.1184936 ,
        0.42608762,  0.09344044,  0.23876254,  0.03202659, -0.48474556,
        0.20207682,  0.3591411 ,  0.14153151, -0.59158474,  0.22894031,
       -0.50379354,  0.26142266,  0.13090676,  0.2817624 ,  0.18732384,
       -0.70734394, -0.17352863, -0.41382965, -0.07756615, -0.27795506,
        0.29656872, -0.2805533 , -0.14593855, -0.26067066, -0.3245186 ,
        0.06118457, -0.43256775, -0.01767822, -0.2653519 , -0.36534983,
        0.6677341 ,  0.06900666,  0.1283574 ,  0.55653936,  0.3253815 ,
       -0.6583108 ,  0.3805514 ,  0.22883272,  0.03484762,  0.16642113,
        0.1955169 ,  0.19038297, -0.45978984,  0.5591574 ,  0.20669614,
       -0.25710267, -0.63774097, -0.19494958,  0.2651378 ,  0.06040186,
        0.28029934, -0.19887646, -0.41886365,  0.49031952, -0.3546757 ,
       -0.3874937 , -0.2980653 ,  0.46184003,  0.48511234,  0.3602015 ,
        0.01042802, -0.24521618,  0.9268969 ,  0.05290376, -0.50

In [32]:
model_to = pd.DataFrame(model.wv['warming']).to_csv("model1.csv", header=None, index=None)

In [34]:
m1 = pd.read_csv("model1.csv")
m1

,-0.084031515
0,-0.432678
1,-0.556972
2,-0.078228
3,0.118494
4,0.426088
...,...
94,0.358024
95,-0.372671
96,0.740263
97,-0.265825


In [26]:
import numpy
numpy.savetxt("model.csv", model.wv['warming'], delimiter=",")

In [27]:
m = pd.read_csv("model.csv")

In [31]:
np.array(m)

99